In [2]:
import torch
from torch import nn


class Network(nn.Module):
    def __init__(self, num_hidden=2, dropout=0., in_feats=2048, num_classes=2):
        super(TheFuckingSledge, self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv1d(1, 8, kernel_size=7, padding=3),
            nn.BatchNorm1d(8),
            nn.ReLU()
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(8, 16, kernel_size=5, stride=2, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(16, 16, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        self.avgpool = nn.AvgPool1d(2, stride=2)
        self.block4 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU()
        )
        self.fc = nn.Linear(3840, 2)
        
    def forward(self, x):
        x = self.block1(x.unsqueeze(1))
        x = self.block2(x)
        x = self.block3(x)
        x = self.avgpool(x)
        x = self.block4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [3]:
Network(dropout=0.5)

TheFuckingSledge(
  (block1): Sequential(
    (0): Conv1d(1, 8, kernel_size=(7,), stride=(1,), padding=(3,))
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Conv1d(8, 16, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block3): Sequential(
    (0): Conv1d(16, 16, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (avgpool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (block4): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (fc): Linear(in_features=3840, out_features=2, bias=True)
)

In [4]:
import numpy as np

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

class ConductanceDataset(Dataset):
    def __init__(self, all_conductance: np.array, in_or_out: np.array, transform=None, target_transform=None):
        super(ConductanceDataset, self).__init__()
        
        self.all_conductance = all_conductance
        self.in_or_out = in_or_out
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.conductance)
    
    def __getitem__(self, ix):
        conductance = self.all_conductance[ix]
        label = self.in_or_out[ix]
        
        if self.transform is not None:
            conductance = self.transform(conductance)
        if self.target_transform is not None:
            label = self.target_transform(label)
            
        return {'ix': ix, 'conductance': conductance, 'label': label}

In [10]:
import os
import numpy as np

CONDUCTANCE_DIR = './zf_net_conductance'
ID_all_conductance = None
OOD_all_class_conductances = None
# layers = ['conv1', 'conv2', 'conv3', 'conv4']
layers = ['conv1']

for layer in layers:
    ID_val_conductance = np.load(os.path.join(CONDUCTANCE_DIR, f'val_attri_{layer}_.npy')).astype('float32')

    ID_test_class_conductances = np.load(os.path.join(CONDUCTANCE_DIR, f'test-id_attrib_{layer}.npy')).astype('float32')
    ID_test_labels = np.load(os.path.join(CONDUCTANCE_DIR, f'test-id_labels_{layer}.npy')).astype('float32')
    ID_test_conductance = np.concatenate((ID_test_class_conductances[0, ID_test_labels == 0], ID_test_class_conductances[1, ID_test_labels == 1]), axis=0)

#     reorder_test_id = np.concatenate((np.where(is_a_hammer_two == 0)[0], np.where(is_a_hammer_two == 1)[0]))
    
    ID_conductance = np.concatenate((ID_val_conductance, ID_test_conductance), axis=0)
    OOD_class_conductances = np.load(os.path.join(CONDUCTANCE_DIR, f'test-ood_attrib_{layer}.npy'))[0].astype('float32')
    if ID_all_conductance is not None:
        ID_all_conductance = np.concatenate((ID_all_conductance, ID_conductance), axis=-1)
        OOD_all_class_conductances = np.concatenate((OOD_all_class_conductances, OOD_class_conductances), axis=-1)
    else:
        ID_all_conductance = ID_conductance
        OOD_all_class_conductances = OOD_class_conductances

In [9]:
ID_conductance = ID_all_conductance
OOD_class_conductances = OOD_all_class_conductances
print(ID_conductance.shape)
print(OOD_class_conductances.shape)

(3000, 96)
(8000, 96)


In [7]:
from sklearn.model_selection import train_test_split

idx = np.arange(len(ID_conductance))
ID_train_idx, ID_test_idx = train_test_split(idx, test_size=0.1)
ID_train_idx, ID_val_idx = train_test_split(ID_train_idx, test_size=0.1481)

ID_train = ID_conductance[ID_train_idx]
ID_val = ID_conductance[ID_val_idx]
ID_test = ID_conductance[ID_test_idx]

idx = np.arange(len(OOD_class_conductances))
OOD_train_idx, OOD_test_idx = train_test_split(idx, test_size=0.66)
OOD_train_idx, OOD_val_idx = train_test_split(OOD_train_idx, test_size=0.15)

OOD_train = other_tools[OOD_train_idx]
OOD_val = other_tools[other_tools_val_idx]
OOD_test = other_tools[other_tools_test_idx]

print(f"data_distribution: \n"
    f"hammers: {len(hammers_train)} {len(hammers_val)} {len(hammers_test)},\n"
    f"other_tools: {len(other_tools_train)} {len(other_tools_val)} {len(other_tools_test)}")

data_distribution: 
hammers: 2300 400 300,
other_tools: 2312 408 5280


In [8]:
from torch.utils.data import DataLoader

training_tools = np.concatenate((hammers_train, other_tools_train), axis=0)
training_is_a_hammer = np.concatenate((np.ones(len(hammers_train)), np.zeros(len(other_tools_train))), axis=0).astype('int')

training_hammersack = TheHammerSack(training_tools, training_is_a_hammer)
training_hammersack = DataLoader(training_hammersack, batch_size=64, pin_memory=True, drop_last=False, shuffle=True)

In [9]:
validation_tools = np.concatenate((hammers_val, other_tools_val), axis=0)
validation_is_a_hammer = np.concatenate((np.ones(len(hammers_val)), np.zeros(len(other_tools_val))), axis=0).astype('int')

validation_hammersack = TheHammerSack(validation_tools, validation_is_a_hammer)
validation_hammersack = DataLoader(validation_hammersack, batch_size=64, pin_memory=True, drop_last=False, shuffle=False)

In [10]:
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = 'cuda:1'
# the_hammer = TheHammer(dropout=0.5).to(device)
the_hammer = TheFuckingSledge(dropout=0.5).to(device)
criterion = CrossEntropyLoss(reduction='mean').to(device)
optimizer = SGD(params=the_hammer.parameters(), lr=0.01, weight_decay=5e-5)
scheduler = ReduceLROnPlateau(optimizer, patience=10, cooldown=10, factor=0.1, verbose=True)

In [11]:
import torch
from collections import defaultdict
from tqdm import tqdm
import json

In [12]:
from inferno.utils.train_utils import AverageMeter
from sklearn.metrics import classification_report

best_loss = np.inf
best_acc = 0
times_it_was_wrong = np.zeros(len(validation_hammersack.dataset), dtype='int')
for epoch in range(100):
    print(f'epoch: {epoch}')
    
    the_hammer.train()
    train_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for sample in training_hammersack:
        ix = sample['ix']
        tools = sample['tool']
        is_it_a_hammer = sample['is_it_a_hammer']
        
        tools = tools.to(device)
        is_it_a_hammer = is_it_a_hammer.to(device)
        
        logits = the_hammer(tools)
        loss = criterion(logits, is_it_a_hammer)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        all_labels.extend(is_it_a_hammer.cpu().detach().numpy().tolist())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        train_trackers['loss'].update(loss.item())
        
    print('train: {}'.format({k: v.avg for k, v in train_trackers.items()}))
    print(f'train acc: {classification_report(y_true=all_labels, y_pred=all_preds, output_dict=True)["accuracy"]}')
        
    the_hammer.eval()
    val_trackers = defaultdict(AverageMeter)
    all_labels, all_preds = [], []
    for sample in validation_hammersack:
        ix = sample['ix']
        tools = sample['tool']
        is_it_a_hammer = sample['is_it_a_hammer']  
          
        tools = tools.to(device)
        is_it_a_hammer = is_it_a_hammer.to(device)
        
        with torch.no_grad():
            logits = the_hammer(tools)
          
        preds = logits.argmax(dim=1)
        wrong_mask = (preds != is_it_a_hammer)
        times_it_was_wrong[ix.numpy()[wrong_mask.cpu().numpy()]] += 1
            
        loss = criterion(logits, is_it_a_hammer)
        
        all_labels.extend(is_it_a_hammer.cpu().detach().numpy().tolist())
        all_preds.extend(logits.argmax(dim=1).detach().cpu().numpy().tolist())
        val_trackers['loss'].update(loss.item())
    
    print('val: {}'.format({k: v.avg for k, v in val_trackers.items()}))
    val_acc = classification_report(y_true=all_labels, y_pred=all_preds, output_dict=True)["accuracy"]
    print(f'val acc: {val_acc}')
    
    val_loss = val_trackers['loss'].avg
    scheduler.step(val_loss)
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(the_hammer.state_dict(), 'sledge_best.pth')
    if val_acc > best_acc:
        best_acc = val_acc

print(np.unique(times_it_was_wrong, return_counts=True))

epoch: 0
train: {'loss': 0.4314435700031176}
train acc: 0.8185169124024284
val: {'loss': 0.4623537648182649}
val acc: 0.7586633663366337
epoch: 1
train: {'loss': 0.31096838475906685}
train acc: 0.8809627059843885
val: {'loss': 1.5230983525084762}
val acc: 0.49504950495049505
epoch: 2


/home/jhaas/Python_Envs36/default/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


train: {'loss': 0.27480545935973727}
train acc: 0.8905030355594102
val: {'loss': 0.24665581377652976}
val acc: 0.8935643564356436
epoch: 3
train: {'loss': 0.2216960518327478}
train acc: 0.9121856027753686
val: {'loss': 0.3296647660720807}
val acc: 0.8415841584158416
epoch: 4
train: {'loss': 0.19771912561295785}
train acc: 0.9241110147441457
val: {'loss': 0.20065097281566033}
val acc: 0.926980198019802
epoch: 5
train: {'loss': 0.17600635326888464}
train acc: 0.9371205550737207
val: {'loss': 0.30204353682123697}
val acc: 0.8663366336633663
epoch: 6
train: {'loss': 0.16132592726243686}
train acc: 0.9386383347788378
val: {'loss': 0.19129512860224798}
val acc: 0.926980198019802
epoch: 7
train: {'loss': 0.145365282229773}
train acc: 0.9507805724197745
val: {'loss': 0.1975262162203972}
val acc: 0.9047029702970297
epoch: 8
train: {'loss': 0.14184693389967695}
train acc: 0.9466608846487424
val: {'loss': 0.16186493864426246}
val acc: 0.9356435643564357
epoch: 9
train: {'loss': 0.1276847453762407

In [13]:
import matplotlib.pyplot as plt
plt.bar(range(len(times_it_was_wrong)), times_it_was_wrong)
plt.show()

In [14]:
best_loss

0.1358512949485045

In [15]:
best_acc

0.9492574257425742

In [16]:
k = 10
top_k_idx = np.argpartition(times_it_was_wrong, -k)[-k:]

top_k_id_idx = top_k_idx[top_k_idx < len(hammers_val_idx)]
top_k_ood_idx = top_k_idx[top_k_idx >= len(hammers_val_idx)] - len(hammers_val_idx)

val_id_indices = hammers_val_idx
val_ood_indices = other_tools_val_idx

In [17]:
print(val_id_indices[top_k_id_idx])
print(val_ood_indices[top_k_ood_idx])

[1395 1207]
[7881 6770 4582 6143 6918 2254 5661 4150]


In [18]:
import os

val_id_image_index = np.concatenate(
    (np.load(os.path.join('out', 'val-id_index.npy')),
     np.load(os.path.join('out', 'test-id_index.npy')))
)[val_id_indices[top_k_id_idx]]
val_ood_image_index = np.load(os.path.join('out', 'test-ood_index.npy'))[val_ood_indices[top_k_ood_idx]]

In [19]:
val_id_image_index

array([1958,  964])

In [20]:
val_ood_image_index

array([9853, 8446, 5739, 7668, 8637, 2817, 7074, 5181])

In [21]:
from torchvision import datasets

dataset_name = 'CIFAR10'
data_root = './data'
dataset = getattr(datasets, dataset_name)(root=data_root, train=True, download=True)

Files already downloaded and verified


In [22]:
for ix in val_id_image_index:
    print(dataset[ix])

(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E4116CBA8>, 3)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E4116CC18>, 9)


In [23]:
for ix in val_ood_image_index:
    print(dataset[ix])

(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175320>, 0)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175390>, 6)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175320>, 4)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175390>, 8)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175320>, 8)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175390>, 8)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175320>, 2)
(<PIL.Image.Image image mode=RGB size=32x32 at 0x7F3E41175390>, 1)
